In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras

In [ ]:
df = pd.read_csv('train_2022-01-05 15-08-53.mp4.csv', index_col = False)
df = df.drop(df[(df['Airspeed(Ind)'] < 0) | (df['Airspeed(Ind)'] > 120)].index)
df['Airspeed(Ind)'] = df['Airspeed(Ind)'].astype(int)

label = 'Airspeed(Ind)'
image_path = 'image_path'

In [ ]:
# classes = []

# for i in df[label]:
#     classes.append(i-75)

# df['classes'] = classes

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_valid, y_train, y_valid = train_test_split(df[[image_path]], df[label], test_size = 0.2, random_state= 42) #, stratify= df[label]) 

In [ ]:
from sklearn.utils import class_weight
class_weights = dict(zip(np.unique(y_train),
                         class_weight.compute_class_weight('balanced', np.unique(y_train), y_train))) 

In [ ]:
x = class_weights.keys()
y = class_weights.values()
plt.bar(x,y, 0.5)
plt.xlabel('Categories')
plt.ylabel("Values")
plt.title('Class Weightage')
plt.legend('')
plt.show()

In [ ]:
import os
from scipy import stats
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, UpSampling2D,Dropout, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
from tensorflow.keras import optimizers , layers, applications
from scipy import stats
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, UpSampling2D,Dropout, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.vis_utils import plot_model
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, save_model
import efficientnet.tfkeras as efn

In [ ]:
df_train = pd.concat([X_train,y_train],axis=1)
df_train[label] = df_train[label].astype(str) 
df_valid = pd.concat([X_valid,y_valid],axis=1)
df_valid[label] = df_valid[label].astype(str) 

In [ ]:
EXPERIMENT_NO = 7
xcol, ycol = image_path, label
TRAINING_DIR = 'gauge_images/airspeed/2022-01-05 15-08-53.mp4'
IMAGE_SIZE = 67
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
EPOCHS = 500
BATCH_SIZE = 256
LR = 0.00001
CHECKPOINT_PATH = "./Weights/{}_Resnet50_xplane_airspeed_weights.hdf5".format(EXPERIMENT_NO)
MONITOR = 'val_loss'

In [ ]:
# Training Generator
train_datagen = ImageDataGenerator(
#     rotation_range=40,
    rescale=1./255,
#     shear_range=0.2,
    zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest',
#     width_shift_range=0.2,
#     height_shift_range=0.2
    brightness_range=[0.1,1.0]
)

train_generator = train_datagen.flow_from_dataframe( 
    df_train,
    TRAINING_DIR, 
    x_col=xcol,
    y_col=ycol,
    class_mode='categorical',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

total_train = df_train.shape[0]
total_validate = df_valid.shape[0]

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    df_valid, 
    TRAINING_DIR, 
    x_col=xcol,
    y_col=ycol,
    class_mode='categorical',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

In [ ]:
pre_trained_model = VGG16(input_shape=INPUT_SHAPE, include_top=False, weights="imagenet", pooling= 'avg')
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# # Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(256, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(120, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr= LR),
              metrics=['accuracy'])

model.summary()

In [ ]:
model_checkpoint_callback = ModelCheckpoint(filepath= CHECKPOINT_PATH, verbose=1, 
                               save_best_only=True, monitor= MONITOR)
early_stop = EarlyStopping(monitor= MONITOR, min_delta=0, patience=5, verbose=0)
reduce_lr = ReduceLROnPlateau(monitor=MONITOR, factor=0.2,
                              patience=3, min_lr=0.000002)

In [ ]:
history = model.fit(
             train_generator,
             class_weight = class_weights,
             validation_data = validation_generator,
             epochs = 1000,
             callbacks = [reduce_lr, early_stop, model_checkpoint_callback],
             verbose = 1,
             validation_steps=total_validate//BATCH_SIZE,
             steps_per_epoch=total_train//BATCH_SIZE)